In the 'markdown cell' below  replace the `???` with the names of those in your group.

???

# Assignment (part 1): classification of cell morphological changes with LeNet
_by Phil Harrison (February 2021)_
#### Inspiration
This step by step exercise was inspired by Andrew Ng's **Deep learning** Coursera specialization (https://www.coursera.org/specializations/deep-learning) . This assignment will take you through a machine learning workflow which includes reading datasets, creating and compiling models, training models on datasets, and predicting on the validation sets (for model comparisons). Much of the experience you accumualted during the labs from yesterday and the day before will come in useful here.

#### Datasets
The specific dataset we will use is a subset of the bbbc021v1 MoA dataset (http://mct.aacrjournals.org/content/9/6/1913) available from the Broad Bioimage Benchmark Collection (https://www.nature.com/articles/nmeth.2083). We chose to only use a subset of this data (based on only the six main MoAs) in order that the models we will fit below would not take too long to train.

#### Importance
After you've completed this assignment you will know how to develop and utilize advanced machine learning models (in this case convlutional neural networks (CNNs) applied to high content cell images). Traditional approaches for classyfing such biological cell-images involve complex workflows, with many steps requiring manual implementation. The more modern neural network approach (made possible through the better hardware available today, most notably via GPUs) can perform equally well and often better than the traditional approaches. What's more these CNNs, based merely on the pixel-intensities of the images, require significantly less domain expertise.

#### Note 1
Using the entire MoA dataset, transfer learning and data augmentation is a preferable way to more fully explore this data (as was done in the paper written by Alexander Kensert - a former student from the same PB-seq masters course you are now on! (https://journals.sagepub.com/doi/10.1177/2472555218818756)). This paper was discussed during the lectures.

Adding transfer learning to the models will be explored in part 3 of the assignemt. This third part should be attempted by those of you that would like to attain a **VG mark** for the assignment. To attain a **G mark** only parts 1 and 2 need be completed.

#### Note 2
For the assignment we will split the data into a training and validation set and will only optimize our performance on the validation set. Although from the work with the oral cancer data you saw that it is genrally best to keep out a test set on which to make a final evaluation of your chosen model.

## Load packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import random
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
import pandas as pd
from PIL import Image

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
print(tf.__version__)

## Functions
Don't worry too much about the code in the functions below, but you might want to go through when they are called later on so that you roughly understand what they're doing.

In [ ]:
def load_dataset():
    dirname = 'bbbc021v1_images'
    x_orig = np.zeros((660, 256, 256, 3), dtype=np.float32)

    for f in range(x_orig.shape[0]):
        img    = Image.open(dirname + '/bbbc021v1_%s.png' % str(f))
        img    = np.array(img)
        x_orig[f] = img

    labels = pd.read_csv('bbbc021v1_labels.csv',
                          usecols=["compound", "concentration", "moa"],
                          sep=";")
    y_orig = np.array(labels['moa'])

    return x_orig, y_orig

def convert_to_one_hot(y, C):
    moa_dict = {'Aurora kinase inhibitors': 0, 'Cholesterol-lowering': 1,
                'Eg5 inhibitors': 2, 'Protein synthesis': 3, 'DNA replication': 4, 'DNA damage': 5}

    y = np.asarray([moa_dict[item] for item in y])
    y = np.eye(C)[y]
    y = y.astype('float32')

    return y

def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15, 5), facecolor='w')
    ax = fig.add_subplot(131)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['train', 'valid'], loc='upper right')
    
    ax = fig.add_subplot(132)
    ax.plot(np.log(model_history.history['loss']))
    ax.plot(np.log(model_history.history['val_loss']))
    ax.set(title=model_name + ': Log model loss', ylabel='Log loss', xlabel='Epoch')
    ax.legend(['Train', 'Test'], loc='upper right')    

    ax = fig.add_subplot(133)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['train', 'valid'], loc='upper right')
    plt.show()
    plt.close()

def plot_confusion_matrix(cm, classes, model_name,
                          cmap=plt.cm.Blues):
    title = model_name + ': Confusion Matrix'
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
def valid_evaluate(model, model_name):
    y_pred = model.predict(X_valid)
    y_pred = y_pred.argmax(axis=-1)
    y_true = Y_valid.argmax(axis=-1)
    
    class_names = ['Aur', 'Ch', 'Eg5', 'PS', 'DR', 'DS']
    cnf_matrix = confusion_matrix(y_true, y_pred)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(15,5), facecolor='w')
    plot_confusion_matrix(cnf_matrix, classes=class_names, model_name=model_name)
    plt.close()
    
    print('')
    print('classification report for validation data:')
    print(classification_report(y_true, y_pred, digits=3))

## Read dataset

In [ ]:
X_orig, y_orig = load_dataset()
print("Shape of y_orig = " + str(y_orig.shape))

## One hot encoding of targets and standardization of images

In [ ]:
Y = convert_to_one_hot(y_orig, 6)
X = X_orig/255.

print("Shape of Y_orig = " + str(Y.shape))

## Split data into training and validation sets
We will use 500 of the images for training and the remaining 160 for validation.

In [ ]:
n_train = 500

random.seed(5026)
indices = np.arange(len(Y))
random.shuffle(indices)

X_train, X_valid = X[indices[:n_train]], X[indices[n_train:]]
Y_train, Y_valid = Y[indices[:n_train]], Y[indices[n_train:]]

## Plot samples of the data

In [ ]:
# Run cell to plot few example images of the training set
fig, ax = plt.subplots(2, 3, figsize=(14, 10))
fig.suptitle("Examples of training images", fontsize=20)
axes = ax.ravel()
for i in range(len(axes)):
    idx = np.where(np.argmax(Y_train, axis=1) == i)[0]
    s_idx = np.random.choice(idx)
    img = (X_train[s_idx]*255).astype("uint8")
    axes[i].set_title(y_orig[indices][s_idx], fontsize=14)
    axes[i].imshow(img)
    axes[i].set_axis_off()
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()
plt.close()